## Unrolling a QASM program
- We take the example from the `openqasm3` repository and produce the unrolled program.  

In [ ]:
%pip install pyqasm --quiet

In [ ]:
import pyqasm 
pyqasm.__version__

In [ ]:
from pyqasm import loads, dumps

- [`Adder.qasm`](https://github.com/openqasm/openqasm/blob/9150b157df801702b02ac7c14979ace8aba53a85/examples/adder.qasm)

In [ ]:
adder_qasm_str = """
/*
 * quantum ripple-carry adder
 * Cuccaro et al, quant-ph/0410184
 */
OPENQASM 3.0;
include 'stdgates.inc';

gate majority a, b, c {
    cx c, b;
    cx c, a;
    ccx a, b, c;
}

gate unmaj a, b, c {
    ccx a, b, c;
    cx c, a;
    cx a, b;
}

qubit[1] cin;
qubit[4] a;
qubit[4] b;
qubit[1] cout;
bit[5] ans;
array[bool, 4] a_in = {0,0,0,1};  // a = 0001
array[bool, 4] b_in = {1,1,1,1}; // b = 1111
// initialize qubits
reset cin;
reset a;
reset b;
reset cout;

// set input states
for int i in [0: 3] {
  if(a_in[i]) x a[i];
  if(b_in[i]) x b[i];
}

// add a to b, storing result in b
majority cin[0], b[0], a[0];
for int i in [0: 2] { majority a[i], b[i + 1], a[i + 1]; }
cx a[3], cout[0];
for int i in [2: -1: 0] { unmaj a[i],b[i+1],a[i+1]; }
unmaj cin[0], b[0], a[0];
measure b[0:3] -> ans[0:3];
measure cout[0] -> ans[4];
"""


In [ ]:
module = loads(adder_qasm_str)
module.validate()
module.unroll()

In [ ]:
dumps(module).splitlines()